In [9]:
import argparse
import torch
import numpy as np
from typing import Optional
import sys
sys.path.append('../')
from acds.archetypes import RandomizedOscillatorsNetwork
from att_dim_experiments.utils import  load_results
import pandas as pd
from pandas import DataFrame
from skdim.id import lPCA, CorrInt
from tqdm import tqdm
tqdm.pandas()

In [10]:

def compute_corr_dim(trajectory: np.ndarray, k1=10,  k2=20, transient=1000) -> Optional[list[float]]:
    corr_dim_values = []
    for i in range(trajectory.shape[1]): # for each module in the network
        corr_dim_estimator = CorrInt(k1=k1, k2=k2)
        try:
            traj_i = trajectory[:, i]
            corr_dim = corr_dim_estimator.fit_transform(traj_i)
            corr_dim_values.append(corr_dim)
        except Exception as e:
            print(f"Error computing correlation dimension: {e}")
            return None
    return corr_dim_values



In [11]:

trajectory_path = "../trajectories/cycle_trajectories_collection.pkl"

data = load_results(trajectory_path)
print(type(data))

#df = DataFrame(data)
meta = dict(data['metadata'])
df = DataFrame([
{**t, **h}
for t, h in zip(data["trajectories"], data["hyperparameters"])
])
corr_dim = lambda traj: compute_corr_dim(traj)
print(df['h'][0].shape)
df['corr_dim'] = df['h'].progress_map(corr_dim)
print(df.head())

df.to_pickle(trajectory_path.replace('.pkl', '_with_corrdim.pkl'))

<class 'dict'>
(5001, 16, 128)


 26%|██▌       | 311/1200 [09:56<28:24,  1.92s/it]


KeyboardInterrupt: 